# CVD-Net Database Pipeline: Tutorial 

This is a work in progress tutorial notebook. It guides a user in how to create a database schema in postgreSQL (**psql**), and then uses **Python** functions to load in a data dictionary from a CSV file into a **psql** table. Below are some brief notes on how to this works - more context will be added soon. 

## Assumptions

1. We are writing and executing this code on our local machines (**MacOS**), with **psql** installed via [Homebrew](https://www.postgresql.org/download/macosx/). It is important to note that the database set-up and interaction could be different once on the TRE. 
2. We are working within a virtual env (`conda create --name cvdnet python=3.13`)
3. We installed these packages in the cvdnet virtual env (`conda install sqlalchemy pandas ipykernel psycopg2 openpyxl`)
4. We are using the **Visal Studio Code** IDE with [PostgreSQL Explorer extension](https://marketplace.visualstudio.com/items?itemName=ckolkman.vscode-postgres) - not essential, but allows for nice visualisation of the database. 
5. We created the schema on the shell terminal, and checked we could view the tables:
   - `dropdb CVD-Net` 
   - `createdb CVD-Net` 
   - `psql -f pipeline/CVD-net_consolidated_DDL.sql CVD-Net`
6. Set up the database connection in the PostgreSQL Explorer extension, in order to visualise the database and query the tables or the views (right click). 
   - Click the plus symbol to 'Add Connection'
   - hostname: localhost
   - user: your postgres username
   - password: left blank for now
   - port: use default
   - ssl connection: use default
   - database to connect to: CVD-Net

In [ ]:
import pandas as pd
import sqlalchemy

In [ ]:
# Running main.py will import all other parts of the pipeline
# The database engine is generated upon loading
# All other functions can be called after running this script

%run main.py

## ASPIRE data

In [ ]:
# load in ASPIRE data dictionary 
 
ASPIRE_dict = metadata.load_dictionary_file(path_to_file='../dummy_data/ASPIRE_dictionary_to_template.csv')

In [ ]:
# view ASPIRE data dictionary df

ASPIRE_dict.df

In [ ]:
# insert variables into the database (respond to the prompts with 'y')

metadata.insert_variables(engine=engine,formatted_dictionary=ASPIRE_dict)

In [ ]:
# See the metadata that you just inserted into the database
# can also do this from the SQL extension 

pd.read_sql(sql = "SELECT * FROM cvdnet_consolidated.view_metadata;",con = engine)


In [ ]:
# generate ASPIRE dummy data (could change to a function for ease of use) - select 200-500 subjects for testing

%run ../dummy_data/generate_ASPIRE_dummy_data.py

In [ ]:
# transform raw (dummy) data to template format (this is the only function that needs to be replaced for a new dataset)

transform_raw_data.aspire_core_to_template(path_to_file='../dummy_data/ASPIRE_dummy_data.xlsx',output_directory='../dummy_data/')


In [ ]:
# load in transformed measurement (dummy) data
# You may need to update the file path before running as the file you generated will likely have a different date (should be printed just above this box)!

ASPIRE_meas = measurements.load_measurement_file(path_to_file="../dummy_data/ASPIRE_dummy_data_TRANSFORMED_2025-02_18_160046.csv")
# insert measurements 

In [ ]:
ASPIRE_meas.df

In [ ]:
# Insert the measurements (and subjects) into the database

measurements.insert_measurements(formatted_meas=ASPIRE_meas, engine=engine)

In [ ]:
# See the subjects and measurements that you just inserted into the database

pd.read_sql(sql = "SELECT * FROM cvdnet_consolidated.view_subject_measurements;",con = engine)

## FIT-PH data

In [ ]:
# Now follow the same workflow for loading the FIT-PH data
# Starting with loading in the variables
FITPH_dict = metadata.load_dictionary_file(path_to_file="../dummy_data/FIT-PH_dictionary_to_template.csv")
metadata.insert_variables(formatted_dictionary=FITPH_dict, engine=engine)

In [ ]:
# Preview the new FIT-PH metadata
# You can see that the variables are from two sensors (in the category_level_2 column)
pd.read_sql("SELECT * FROM cvdnet_consolidated.view_metadata WHERE dataset_name = 'FIT-PH';", con=engine)

In [ ]:
# generate FIT-PH dummy data (could change to a function for ease of use) - select 10 subjects for testing as this dataset is more dense so less subjects used

%run ../dummy_data/generate_FIT-PH_dummy_data.py

In [ ]:
# Now transform the dummy data into the template format using a function developed for this FIT-PH sensor data
transform_raw_data.fitph_sensor_to_template(path_to_file="../dummy_data/FIT-PH_dummy_data.xlsx", output_directory="../dummy_data/")

In [ ]:
# Then load in the transformed file
# You may need to update the file path name before running as the file you generated will likely have a different date (should be printed just above this box)!
FITPH_meas = measurements.load_measurement_file(path_to_file="../dummy_data/FIT-PH_dummy_data_TRANSFORMED_2025-02_18_161214.csv")

In [ ]:
# Insert measurements into the database
# As it needs to validate a lot of measurements it can be a bit slow (approx 3-6 mins on a laptop - should be faster on TRE, and VMs can be changed if proving to be too slow)
measurements.insert_measurements(formatted_meas=FITPH_meas, engine=engine)

In [ ]:
# Have a look at our new measurements
pd.read_sql(sql = "SELECT * FROM cvdnet_consolidated.view_subject_measurements WHERE dataset_name = 'FIT-PH';",con = engine)

## Preparing for releasing data to the Insights TRE

In [ ]:
# Test preparing for an export of the data for the Insights TRE
# This script will make a second de-identified version of the schema and prepare a database dump of it for export
# Script is a shell script so will need to be run from the terminal
    # ./prepare_deidentified_database.sh

Then check the PostgreSQL Explorer extension and you will see the two schemas (may need to right click and refresh on the database name first)